In [45]:
import nltk 
import nltk.corpus as corpus
import pandas as pd

In [46]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [47]:
data = pd.read_excel("Input.xlsx")
data

,URL_ID,URL
0,123.0,https://insights.blackcoffer.com/rise-of-telem...
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...
4,432.0,https://insights.blackcoffer.com/rise-of-telem...
...,...,...
109,50921.0,https://insights.blackcoffer.com/coronavirus-i...
110,51382.8,https://insights.blackcoffer.com/coronavirus-i...
111,51844.6,https://insights.blackcoffer.com/what-are-the-...
112,52306.4,https://insights.blackcoffer.com/marketing-dri...


In [48]:
pip install beautifulsoup4 requests pandas

Note: you may need to restart the kernel to use updated packages.


In [49]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def extract_article_text(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  
        soup = BeautifulSoup(response.content, 'html.parser')
        article_title = soup.find('title').get_text()
        article_text = ' '.join([p.get_text() for p in soup.find_all('p')])
        return article_title, article_text
    except requests.exceptions.RequestException as e:
        print(f"Error accessing {url}: {e}")
        return None, None

input_df = pd.read_excel('Input.xlsx')

for index, row in input_df.iterrows():
    url_id = row['URL_ID']
    url = row['URL']
    article_title, article_text = extract_article_text(url)
    if article_text is not None:
        with open(f'extracted_test/{url_id}.txt', 'w', encoding='utf-8') as file:
            file.write(f"{article_title}\n\n{article_text}")

print("Data extraction completed.")


Error accessing https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/: 404 Client Error: Not Found for url: https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/
Error accessing https://insights.blackcoffer.com/covid-19-environmental-impact-for-the-future/: 404 Client Error: Not Found for url: https://insights.blackcoffer.com/covid-19-environmental-impact-for-the-future/
Data extraction completed.


In [50]:
pip install bs4

Note: you may need to restart the kernel to use updated packages.


In [51]:
pip install nltk textblob pandas

Note: you may need to restart the kernel to use updated packages.


In [53]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import re
from textblob import TextBlob
import pandas as pd

nltk.download('punkt')
nltk.download('stopwords')

def clean_text(text):
    # Tokenize the text
    words = word_tokenize(text)

    # Remove stop words
    stop_words = set(stopwords.words("english"))
    words = [word.lower() for word in words if word.isalpha() and word.lower() not in stop_words]

    # Remove punctuations
    words = [re.sub(r'[^\w\s]', '', word) for word in words]

    return words


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [54]:
def count_syllables(word):
    # Simple syllable count (excluding some exceptions)
    vowels = "aeiouy"
    count = 0
    word = word.lower()

    if word[0] in vowels:
        count += 1

    for i in range(1, len(word)):
        if word[i] in vowels and word[i - 1] not in vowels:
            count += 1

    if word.endswith("e"):
        count -= 1

    if word.endswith("le"):
        count += 1

    if count == 0:
        count += 1

    return count

In [55]:
def calculate_variables(text):
    words = clean_text(text)
    num_words = len(words)
    num_sentences = len(sent_tokenize(text))
    num_complex_words = sum(1 for word in words if count_syllables(word) > 2)

    avg_sentence_length = num_words / num_sentences
    percentage_complex_words = num_complex_words / num_words
    fog_index = 0.4 * (avg_sentence_length + percentage_complex_words)

    avg_words_per_sentence = num_words / num_sentences
    complex_word_count = num_complex_words
    syllable_per_word = sum(count_syllables(word) for word in words) / num_words

    # Sentiment Analysis using TextBlob
    blob = TextBlob(text)
    positive_score = len([s for s in blob.words if s.lower() in positive_words])
    negative_score = len([s for s in blob.words if s.lower() in negative_words])

    polarity_score = (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)
    subjectivity_score = (positive_score + negative_score) / (num_words + 0.000001)

    # Personal Pronouns count using regex
    personal_pronouns = len(re.findall(r'\b(?:I|we|my|ours|us)\b', text, flags=re.IGNORECASE))

    avg_word_length = sum(len(word) for word in words) / num_words

    return [positive_score, negative_score, polarity_score, subjectivity_score,
            avg_sentence_length, percentage_complex_words, fog_index,
            avg_words_per_sentence, complex_word_count, num_words,
            syllable_per_word, personal_pronouns, avg_word_length]


In [56]:
input_df = pd.read_excel('Input.xlsx')

In [57]:
pip install chardet

Note: you may need to restart the kernel to use updated packages.


In [58]:
with open('positive-words.txt', 'r', encoding='cp1252') as file:
    positive_words = set(file.read().splitlines())

with open('negative-words.txt', 'r', encoding='cp1252') as file:
    negative_words = set(file.read().splitlines())

In [59]:
# Create a DataFrame to store the output
output_df = pd.DataFrame(columns=['URL_ID'] + [
    'POSITIVE SCORE', 'NEGATIVE SCORE', 'POLARITY SCORE', 'SUBJECTIVITY SCORE',
    'AVG SENTENCE LENGTH', 'PERCENTAGE OF COMPLEX WORDS', 'FOG INDEX',
    'AVG NUMBER OF WORDS PER SENTENCE', 'COMPLEX WORD COUNT', 'WORD COUNT',
    'SYLLABLE PER WORD', 'PERSONAL PRONOUNS', 'AVG WORD LENGTH'
])


In [60]:
import os
import pandas as pd
results = []

# Iterate over rows in the input dataframe
for index, row in input_df.iterrows():
    url_id = row['URL_ID']
    text_file_name = f"extracted_test/{url_id}.txt"  # Adjust the path to the extracted folder

    # Check if the file exists
    if os.path.exists(text_file_name):
        # Read the text file
        with open(text_file_name, 'r', encoding='utf-8') as file:
            text = file.read()

        # Calculate variables
        variables = calculate_variables(text)

        # Append the results to the list
        results.append({'URL_ID': url_id, **dict(zip(output_df.columns[1:], variables))})
    else:
        print(f"File not found for URL_ID: {url_id}")

# Create a DataFrame from the list of results
output_df = pd.DataFrame(results)

# Save the output DataFrame to an Excel file
output_df.to_excel('Output_final_analysis.xlsx', index=False)

print("Data analysis completed.")


File not found for URL_ID: 11668.0
File not found for URL_ID: 17671.4
Data analysis completed.
